# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments           Customer  \
0  Sep  7 2022  2:44PM  245887        10  0085966643  ISDIN Corporation   
1  Sep  7 2022  2:44PM  245885        10  0085965871  ISDIN Corporation   
2  Sep  7 2022  2:44PM  245885        10  0085965886  ISDIN Corporation   
3  Sep  7 2022  2:44PM  245885        10  0085966006  ISDIN Corporation   
4  Sep  7 2022  2:44PM  245885        10  0085966022  ISDIN Corporation   
5  Sep  7 2022  2:44PM  245885        10  0085966024  ISDIN Corporation   
6  Sep  7 2022  2:44PM  245885        10  0085966048  ISDIN Corporation   
7  Sep  7 2022  2:44PM  245885        10  0085966062  ISDIN Corporation   
8  Sep  7 2022  2:44PM  245885        10  0085966094  ISDIN Corporation   
9  Sep  7 2022  2:44PM  245885        10  0085966104  ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
43  245879       Released          2
44  245881      Executing          1
45  245884       Released          2
46  245885       Released         17
47  245887       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
245879                NaN        NaN       2.0
245881                NaN        1.0       NaN
245884                NaN        NaN       2.0
245885                NaN        NaN      17.0
245887                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245714               22.0        0.0       4.0
245786                7.0        2.0       2.0
245794                0.0        1.0       0.0
245807                0.0        0.0      19.0
245808                0.0        0.0       3.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
245714                 22          0         4
245786                  7          2         2
245794                  0          1         0
245807                  0          0        19
245808                  0          0         3

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               245714         22          0         4
1               245786          7          2         2
2               245794          0          1         0
3               245807          0          0        19
4               245808          0          0         3

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               245714        22                  4
1               245786         7         2        2
2               245794                   1         
3               245807                           19
4               245808                            3

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse             Customer
0   Sep  7 2022  2:44PM  245887        10    ISDIN Corporation
1   Sep  7 2022  2:44PM  245885        10    ISDIN Corporation
18  Sep  7 2022  2:33PM  245884        10          Emerginnova
20  Sep  7 2022  2:24PM  245881        22    NBTY Global, Inc.
21  Sep  7 2022  2:22PM  245879        19  AdvaGen Pharma, Ltd
23  Sep  7 2022  2:09PM  245873        10    ISDIN Corporation
54  Sep  7 2022  2:06PM  245878        12            Hush Hush
55  Sep  7 2022  1:59PM  245877        10          Emerginnova
66  Sep  7 2022  1:49PM  245875        19  AdvaGen Pharma, Ltd
74  Sep  7 2022  1:31PM  245872        19  AdvaGen Pharma, Ltd

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse             Customer Completed  \
0  Sep  7 2022  2:44PM  245887        10    ISDIN Corporation             
1  Sep  7 2022  2:44PM  245885        10    ISDIN Corporation             
2  Sep  7 2022  2:33PM  245884        10          Emerginnova             
3  Sep  7 2022  2:24PM  245881        22    NBTY Global, Inc.             
4  Sep  7 2022  2:22PM  245879        19  AdvaGen Pharma, Ltd             
5  Sep  7 2022  2:09PM  245873        10    ISDIN Corporation             
6  Sep  7 2022  2:06PM  245878        12            Hush Hush             
7  Sep  7 2022  1:59PM  245877        10          Emerginnova             
8  Sep  7 2022  1:49PM  245875        19  AdvaGen Pharma, Ltd             
9  Sep  7 2022  1:31PM  245872        19  AdvaGen Pharma, Ltd             

  Executing Released  
0                  1  
1                 17  
2                  2  
3         1           
4                  2  
5                 31  
6                  1  
7                 11  
8                  8  
9        13       13

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse             Customer Released  \
0  Sep  7 2022  2:44PM  245887        10    ISDIN Corporation        1   
1  Sep  7 2022  2:44PM  245885        10    ISDIN Corporation       17   
2  Sep  7 2022  2:33PM  245884        10          Emerginnova        2   
3  Sep  7 2022  2:24PM  245881        22    NBTY Global, Inc.            
4  Sep  7 2022  2:22PM  245879        19  AdvaGen Pharma, Ltd        2   
5  Sep  7 2022  2:09PM  245873        10    ISDIN Corporation       31   
6  Sep  7 2022  2:06PM  245878        12            Hush Hush        1   
7  Sep  7 2022  1:59PM  245877        10          Emerginnova       11   
8  Sep  7 2022  1:49PM  245875        19  AdvaGen Pharma, Ltd        8   
9  Sep  7 2022  1:31PM  245872        19  AdvaGen Pharma, Ltd       13   

  Executing Completed  
0                      
1                      
2                      
3         1            
4                      
5                      
6                      
7                      
8                      
9        13

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse             Customer Released  \
0  Sep  7 2022  2:44PM  245887        10    ISDIN Corporation        1   
1  Sep  7 2022  2:44PM  245885        10    ISDIN Corporation       17   
2  Sep  7 2022  2:33PM  245884        10          Emerginnova        2   
3  Sep  7 2022  2:24PM  245881        22    NBTY Global, Inc.            
4  Sep  7 2022  2:22PM  245879        19  AdvaGen Pharma, Ltd        2   

  Executing Completed  
0                      
1                      
2                      
3         1            
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Sep  7 2022  2:44PM  245887        10    ISDIN Corporation       1.0   
1  Sep  7 2022  2:44PM  245885        10    ISDIN Corporation      17.0   
2  Sep  7 2022  2:33PM  245884        10          Emerginnova       2.0   
3  Sep  7 2022  2:24PM  245881        22    NBTY Global, Inc.       NaN   
4  Sep  7 2022  2:22PM  245879        19  AdvaGen Pharma, Ltd       2.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        1.0        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse             Customer  Released  \
0  Sep  7 2022  2:44PM  245887        10    ISDIN Corporation       1.0   
1  Sep  7 2022  2:44PM  245885        10    ISDIN Corporation      17.0   
2  Sep  7 2022  2:33PM  245884        10          Emerginnova       2.0   
3  Sep  7 2022  2:24PM  245881        22    NBTY Global, Inc.       0.0   
4  Sep  7 2022  2:22PM  245879        19  AdvaGen Pharma, Ltd       2.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        1.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         5654404     323.0        3.0        7.0
12          737573       2.0        1.0        0.0
15          983214      72.0        8.0       57.0
16          245845       2.0        0.0        0.0
19          983493      24.0       13.0        0.0
20          491723      16.0       14.0        3.0
21          245794       0.0        1.0        0.0
22          245881       0.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  5654404     323.0        3.0        7.0
1        12   737573       2.0        1.0        0.0
2        15   983214      72.0        8.0       57.0
3        16   245845       2.0        0.0        0.0
4        19   983493      24.0       13.0        0.0
5        20   491723      16.0       14.0        3.0
6        21   245794       0.0        1.0        0.0
7        22   245881       0.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     323.0        3.0        7.0
1        12       2.0        1.0        0.0
2        15      72.0        8.0       57.0
3        16       2.0        0.0        0.0
4        19      24.0       13.0        0.0
5        20      16.0       14.0        3.0
6        21       0.0        1.0        0.0
7        22       0.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  323.0
1        12  Released    2.0
2        15  Released   72.0
3        16  Released    2.0
4        19  Released   24.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   12    15   16    19    20   21   22
Status                                                
Completed    7.0  0.0  57.0  0.0   0.0   3.0  0.0  0.0
Executing    3.0  1.0   8.0  0.0  13.0  14.0  1.0  1.0
Released   323.0  2.0  72.0  2.0  24.0  16.0  0.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   12    15   16    19    20   21   22
0          Completed    7.0  0.0  57.0  0.0   0.0   3.0  0.0  0.0
1          Executing    3.0  1.0   8.0  0.0  13.0  14.0  1.0  1.0
2           Released  323.0  2.0  72.0  2.0  24.0  16.0  0.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   12    15   16    19    20   21   22
0  Completed    7.0  0.0  57.0  0.0   0.0   3.0  0.0  0.0
1  Executing    3.0  1.0   8.0  0.0  13.0  14.0  1.0  1.0
2   Released  323.0  2.0  72.0  2.0  24.0  16.0  0.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()